In [1]:
import numpy as np
import cmath
import sys
import random
import math
from sklearn.linear_model import Lasso
np.set_printoptions(threshold=sys.maxsize)
def array(*args, **kwargs):
    kwargs.setdefault("dtype", np.complex)
    return np.array(*args, **kwargs)

In [2]:
AngRes = 2*cmath.pi/36
Dres = 0.03
alpha = 0.001

In [3]:
def green(a, b, f):
    dis = np.sqrt(np.sum(np.square(a-b)))+0j
    k = (2*cmath.pi*f)/(343)
    return (cmath.exp(-1j*k*dis)/dis)
def zone(R,Theta,r,N):
    c = np.zeros((2,), dtype = 'complex')
    c[0],c[1] = R*cmath.cos(Theta), R*cmath.sin(Theta)
    ans = np.zeros((N-N%36,2), dtype = 'complex')
    for i in range(int(N/36)):
        for j in range(36):
            ans[i*36 + j][0], ans[i*36 + j][1] = c[0]+((r)/int(N/36))*(i+1)*cmath.cos((2*cmath.pi*(j))/36),c[1]+(r/int(N/36))*(i+1)*cmath.sin((2*cmath.pi*(j))/36)
    return ans
def sploc(R,n):
    ang = (2*cmath.pi)/n
    ans = np.zeros((n,2), dtype = 'complex')
    for i in range(n):
        ans[i][0], ans[i][1] = R*cmath.cos(ang*i), R*cmath.sin(ang*i)
    return ans
def Hq(m,n,l,R,Rz,r,theta,f):
    m = m - m%36
    sp = sploc(R,l)
    zp = np.zeros((n*m,2), dtype = 'complex')
    #below can be sped up by concat
    for i in range(n):
        zz = zone(Rz[i],theta[i],r[i],m)
        for j in range(m):
            zp[i*m+j] = zz[j] 
    H = np.zeros((n*m,l), dtype = 'complex')
    for i in range(n*m):
        for j in range(l):
            H[i][j] = green(sp[j], zp[i],f)
    return (H)
def Dsq(m,n,Rz,thetaz,rz,n_z,f):
    R = Rz[n_z]
    theta = thetaz[n_z]
    zp = np.zeros((n*m,2), dtype = 'complex')
    #below can be sped up by concat
    for i in range(n):
        zz = zone(Rz[i],thetaz[i],rz[i],m)
        for j in range(m):
            zp[i*m+j] = zz[j] 
    s = np.zeros((2,), dtype = 'complex')
    s[0], s[1] = R*cmath.cos(theta), R*cmath.sin(theta)
    D = np.zeros((m*n,), dtype = 'complex')
    for i in range(m*n):
        if i < m*n_z or i >= m*(n_z+1) :
            D[i] = green(zp[i],s,f)*alpha
        else:
            D[i] = green(zp[i],s,f)
    return (D)
def Solver(m,n,l,R,Rz,thetaz,rz,f):
    H = Hq(m,n,l,R,Rz,thetaz,rz,f)
    hq = np.zeros((2*m*n,2*l))
    for i in range(n):
        Dt = Dsq(m,n,Rz,thetaz,rz,i,f)
        if i!=0:
            D = np.vstack((D,Dt))
        else:
            D = Dsq(m,n,Rz,thetaz,rz,i,f)
    D = np.transpose(D)
    dsq = np.zeros((2*m*n,n))
    for i in range(m*n):
        for j in range(l):
            hq[2*i][2*j] = np.real(H[i][j])
            hq[2*i][2*j+1] = -np.imag(H[i][j])
            hq[2*i+1][2*j] = np.imag(H[i][j])
            hq[2*i+1][2*j+1] = np.real(H[i][j])
    for j in range(n):
        for i in range(m*n):
            dsq[2*i][j] = np.real(D[i][j])
            dsq[2*i+1][j] = np.imag(D[i][j])
    lm = np.matmul(np.transpose(H),D)
    #alp=(75/4700000)*np.abs(np.max(lm))
    alp=20
    lasso = Lasso(alpha=alp, max_iter=10000000)
    lasso.fit(hq,dsq)
    Wr = np.transpose(lasso.coef_)
    W = np.zeros((l,n),dtype='complex')
    for i in range(l):
        for j in range(n):
            W[i][j] = Wr[2*i][j]+Wr[2*i+1][j]*1j
    
    return W, D

In [4]:
m = 360
n = 5
l = 150
R = 0.2
nf = 3
R_r = 4#Ratio between zone and total radius
o = 0.
for i in range(900): 
    rz = []
    Rz = []
    thetaz = []
    F = []
    for k in range(n):
        rtemp = random.uniform(0,R/R_r)
        rz.append(rtemp)
        flag=-1
        while flag==-1:
            flag=0
            Rtemp = random.uniform(0,R-rtemp)
            phi = random.uniform(-math.pi,math.pi)
            for g in range(k):
                if (rz[g]+rtemp)>math.sqrt(Rz[g]**2+Rtemp**2+2*Rtemp*Rz[g]*math.cos(thetaz[g]-phi)):
                    flag=-1
        Rz.append(Rtemp)
        thetaz.append(phi)
    for i in range(nf):
        f = random.uniform(500,2000)
        F.append(f)
    Wt = np.zeros((l,n))
    j = 0.
    for f in F:
        W, Dt = (Solver(m,n,l,R,Rz,thetaz,rz,f))
        W = (np.abs(W) > 0.01).astype(int)
        Wt = W+Wt
        if(j==0):
            D = Dt
            j = -1
        else:
            D = np.dstack((D,Dt)) 
    Wt = (np.abs(Wt) > 0.01).astype(int)
    W = np.zeros((l,))
    for i in range(np.shape(Wt)[1]):
        W = W+Wt[:,i]
    W = (np.abs(W)>0.1).astype(int)
    if o==0:
        o=1
        Wd = W;
        Rzd = Rz
        rzd = rz
        Dd = np.ravel(D)
        Fd = F
        thetazd = thetaz
    else:
        Wd = np.dstack((Wd,W))
        Rzd = np.dstack((Rzd,Rz))
        rzd = np.dstack((rzd,rz))
        #D = np.reshape(D, (m*n,n,nf,1))
        D = np.ravel(D)
        Dd = np.dstack((Dd,D))
        Fd = np.dstack((Fd,F))
        thetazd = np.dstack((thetazd,thetaz))
np.savetxt('Documents//W.csv', Wd[0], delimiter=',')
np.savetxt('Documents//D.csv', Dd[0], delimiter=',')
np.savetxt('Documents//F.csv', Fd[0], delimiter=',')
np.savetxt('Documents//thetaz.csv', thetazd[0], delimiter=',')
np.savetxt('Documents//radiuszone.csv', rzd[0], delimiter=',')
np.savetxt('Documents//Radialzonedistance.csv', Rzd[0], delimiter=',')

In [39]:
np.shape(Wd[0])

(150, 1000)

In [10]:
sad = np.loadtxt('Documents//D.csv', delimiter=',',dtype='complex')

In [11]:
np.shape(sad)

(27000, 900)